# Translating a Landlab RasterModelGrid into a PyVista StructuredGrid for visualization

*Greg Tucker, CU Boulder, June 2025*

PyVista offers powerful 3D visualization capabilities for geoscientific data. This notebook demonstrates how to use a set of utilities I've written that translate data from a Landlab `RasterModelGrid` and its fields into a PyVista `StructuredGrid`, which can then be visualized interactively directly in a notebook.

The `llpytools` package offers a function called `grid_to_pv` that translates the contents of a Landlab grid and its fields into (generally) two PyVista mesh objects, one for each of the two dual meshes that compose most Landlab grids. In one of the resulting PyVista meshes, the points are Landlab grid *nodes* and the cells are Landlab grid *patches*. In the other, the points are Landlab grid *corners* and the cells are Landlab grid *cells*.

Start with some imports:

In [ ]:
import numpy as np
import pyvista as pv
from llpvtools import grid_to_pv
from landlab import RasterModelGrid, imshow_grid

## Creating a simple example

Here we'll use the same example that appears in one of the PyVista tutorials, but here created initially as a landlab `RasterModelGrid`. We'll assign two fields: the topographic elevation (which will be the $z$ coordinate for our surface), and the gradient. 

To illustrate use of *corners* instead of *nodes*, we'll also assign a field for elevation and gradient values at corners.

In [ ]:
rmg = RasterModelGrid((80, 80), 0.25, xy_of_lower_left=(-10.0, -10.0))

z = rmg.add_zeros("topographic__elevation", at="node")
s = rmg.add_zeros("topographic__gradient", at="node")
zc = rmg.add_zeros("z_at_corners", at="corner")
sc = rmg.add_zeros("gradient_at_corners", at="corner")

# Create a rippled surface, same as in PyVista tutorial
x = rmg.x_of_node
y = rmg.y_of_node
r = np.sqrt(x**2 + y**2)
z[:] = np.sin(r)

# Calculate the gradient in the y-direction
dy, _ = np.gradient(z.reshape((80, 80)))
s[:] = dy.flatten()

# Rippled surface for corners
xc = rmg.x_of_corner
yc = rmg.y_of_corner
rc = np.sqrt(xc**2 + yc**2)
zc[:] = np.sin(rc)

# and gradient
dyc, _ = np.gradient(zc.reshape((79, 79)))
sc[:] = dyc.flatten()

In [ ]:
imshow_grid(rmg, z)

In [ ]:
imshow_grid(rmg, s)

## Visualizing a landlab raster grid as a 2D surface

The function `grid_to_pv` translates a Landlab grid into PyVista. When passed a Landlab `RasterModelGrid`, the function will by default create a pair of 2D PyVista `StructuredGrid` objects, each representing a surface: one formed from grid *nodes* (which are vertices of *patches*), and one formed of grid *corners* (which are vertices of grid *cells*). The grid's fields are included in the data structures as **Data Arrays**. Specifically, any node- or patch-based fields are attached to the node-based PyVista mesh, and any corner- or cell-based fields are attached to the corner-based mesh.

In [ ]:
node_mesh, cnr_mesh = grid_to_pv(
    rmg, field_for_node_z="topographic__elevation", field_for_corner_z="z_at_corners"
)
node_mesh

The above text displays information about one of the two PyVista `StructuredGrid` objects. Note the dimensions 80 x 80 x 1: an 80 x 80 arrangement of grid nodes with one layer.

The mesh can be viewed interactively using its `plot()` method:

In [ ]:
node_mesh.plot(show_edges=True)

To color by a different field, use the `set_active_scalars()` method:

In [ ]:
node_mesh.set_active_scalars('topographic__elevation')
node_mesh.plot(show_edges=True)

We can also deactivate coloring:

In [ ]:
node_mesh.set_active_scalars(None)
node_mesh.plot(show_edges=True)

The above examples use the landlab grid's *nodes* and *patches*. We can also use *corners* and *cells*. The second PyVista mesh returned by `grid_to_pv` consists of 79 by 79 points, which is the shape of *corners* in the Landlab grid (one fewer than nodes in each dimension).

In [ ]:
cnr_mesh

In [ ]:
cnr_mesh.plot(show_edges=True)

## Created a 3D mesh object

To create a 3D mesh object, you can pass the argument `make3d=True` to `grid_to_pv`. The resulting meshes each have two layers. The top layer is the topography (or, more generally, whatever field we happened to use for the $z$ coordinate). The bottom layer can be flat (constant $z$ value), or it can be assigned an array of values or an existing field. The default setting is to make the bottom flat, with a depth $h$ below the lowest surface point, where $h$ equal to half the widest extent of the grid.

In [ ]:
node_3d_mesh, cnr_3d_mesh = grid_to_pv(
    rmg,
    field_for_node_z="topographic__elevation",
    field_for_corner_z="z_at_corners",
    make3d=True
)
node_3d_mesh

In [ ]:
node_3d_mesh.plot(show_edges=True)

### Using a field or array as the base of a 3d mesh

For the bottom layer of a 3d mesh, you can use a constant value, an array (of length equal to the number of nodes or corners), or a field. Here we'll add a field (an inverse of the topography, offset downward) and use it as the base.

In [ ]:
# add a field
rmg.add_field("subsurface_layer", (1.0 - z) - 5.0, at="node")

# convert to meshes, this time using the "subsurface_layer" field as the base for
# the node mesh
node_3d_mesh, cnr_3d_mesh = grid_to_pv(
    rmg,
    field_for_node_z="topographic__elevation",
    field_for_corner_z="z_at_corners",
    make3d=True,
    values_for_node_base="subsurface_layer"
)
node_3d_mesh.plot(show_edges=True)

## Combining node-link-patch and corner-face-cell meshes in a Plotter

The example below plots both of the Landlab dual meshes together: the nodes-links-patches mesh and the corners-faces-cells mesh. In this example, nodes are shown in black, links in red, faces in blue.

In [ ]:
pl = pv.Plotter()
pl.add_mesh(node_mesh, color="red", style="wireframe", line_width=1)
pl.add_mesh(cnr_mesh, color="blue", style="wireframe", line_width=1)
pl.add_points(
    node_mesh.points, color='black', point_size=4, render_points_as_spheres=True
)
pl.show()

## Options

The function signature for `grid_to_pv` illustrates the available optional parameters:

In [ ]:
help(grid_to_pv)

Coming soon (geologically), I hope, will be:

- Functions to do similar translation for other Landlab grid types
- Examples of how to read Landlab output in .vtk format directly into a PyVista data structure
- The ability to translate fields on cells or patches
- Some kind of widget-like tools to flip between different fields
- A way to provide time-animation
- A function to represent a drainage network as a mesh of line segments
- A way to include vector data using arrow glyphs or similar (e.g., flow velocity, sediment transport, etc.)
- EPIC: a reasonably comprehensive visualization tool/widget-set for visualizing landlab output generally